In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')
import numpy as np
import matplotlib.pyplot as plt

from dem import dem4geoid

In [10]:
dem = dem4geoid(bbox=[-4, 4, 2, 12], downloads_dir='downloads')
dem


Creating xarray dataset of DEM over area of interest with buffer of 2 degrees



<xarray.Dataset> Size: 7MB
Dimensions:  (x: 1200, y: 1440)
Coordinates:
  * x        (x) float64 10kB -5.996 -5.988 -5.979 -5.971 ... 3.979 3.988 3.996
  * y        (y) float64 12kB 2.004 2.012 2.021 2.029 ... 13.97 13.98 13.99 14.0
Data variables:
    z        (y, x) float32 7MB -4.807e+03 -4.804e+03 -4.799e+03 ... 260.0 276.0
Attributes:
    Conventions:  COARDS/CF-1.0
    title:        w020n40.nc
    history:      xyz2grd -V -R-020/020/-10/40 -I30c w020n40.Bathymetry.srtm ...
    GMT_version:  4.5.12 [64-bit]
    node_offset:  1

In [ ]:
import os
# os.chdir('..')
# os.getcwd()

In [ ]:
dem['z'].plot(cmap='terrain')
plt.show()

In [ ]:
import os
import requests
from tqdm import tqdm

def download_srtm30plus(url=None, downloads_dir=None, bbox=None):
    '''
    Parameters
    ----------
    url           : url of the srtm30plus tile
    downloads_dir : directory to download the file to
    bbox          : bbox of the area of interest (to be provided if url is not provided)
    
    Returns
    -------
    fname         : Name of downloaded file
    '''

    if not url:
        url = fetch_url(bbox=bbox)
        
    filename = url.split('/')[-1]
    filepath = os.path.join(downloads_dir, filename) if downloads_dir else filename

    # Check if the file already exists
    if os.path.exists(filepath):
        # Get the expected size of the file
        response_head = requests.head(url, verify=False)
        total_size = int(response_head.headers.get('content-length', 0))

        # Check if the existing file size matches the expected size
        if os.path.getsize(filepath) == total_size:
            print(f'File {filename} already exists and is complete.')
            return filename
        else:
            print(f'File {filename} already exists but is incomplete. Redownloading...')

    if downloads_dir:
        os.makedirs(downloads_dir, exist_ok=True)
        print(f'Downloading {filename} to {downloads_dir} ...')
    else:
        print(f'Downloading {filename} to {os.getcwd()} ...')

    response = requests.get(url, verify=False, stream=True)
    total_size = int(response.headers.get('content-length', 0))

    with open(filepath, 'wb') as f, tqdm(
        desc=filename,
        total=total_size,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as pbar:
        for data in response.iter_content(chunk_size=1024):
            size = f.write(data)
            pbar.update(size)

    return filename
